### Project 3, Part 4
### Sarah Musselwhite
### 11/1/2022

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
with open('/Users/Sarah/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [3]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

In [4]:
## make a movie object using the .Movies function from tmdb
movie = tmdb.Movies(603)
## movie objects have a .info dictionary 
info = movie.info()
info

{'adult': False,
 'backdrop_path': '/l4QHerTSbMI7qgvasqxP36pqjN6.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 59.542,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 174,
   'logo_path': '/IuAlhI9eVC9Z8UQWOIDdWRKSEJ.png'

In [5]:
import os, time,json
import tmdbsimple as tmdb 
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_results_combined.csv.gz']

In [6]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [7]:
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    #get the movie object for the current it
    movie = tmdb.Movies(movie_id)
    
    #save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    # loop through countries in releases
    for c in releases['countries']:
        # if the counrty abbre ==US
        if c['iso_3166_1'] == 'US':
            ## save a certification key in info with the certifications
            info['certification'] = c['certification']
    return info

In [8]:
# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
82974,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
82975,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
82976,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
82977,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


In [9]:
YEARS_TO_GET = [2014,2015,2016,2017,2018,2019]

In [10]:
errors = []

In [11]:
# start of the OUTER LOOP
for YEAR in tqdm_notebook(YEARS_TO_GET, desc= 'YEARS', position = 0):

    # define the JSON file to store results
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

    # check if the file exists
    file_exists = os.path.isfile(JSON_FILE)

    # if the file doesn't exist, create it
    if file_exists == False:
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id':0}],f)

    # save new year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()

    # save movie ids to list
    movie_ids = df['tconst'].copy() #to_list()
    movie_ids

    # load existing data fro json into new file
    previous_df = pd.read_json(JSON_FILE)
    previous_df

    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]


    for movie_id in tqdm_notebook(movie_ids_to_get,
                                 desc=f'Movies from {YEAR}',
                                 position = 1,
                                 leave = True):
        try:
            temp = get_movie_with_rating(movie_id)
            write_json(temp,JSON_FILE)
            time.sleep(0.02)
        except Exception as e: 
            errors.append([movie_id, e])
            

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/6 [00:00<?, ?it/s]

Movies from 2014:   0%|          | 0/4786 [00:00<?, ?it/s]

Movies from 2015:   0%|          | 0/4933 [00:00<?, ?it/s]

Movies from 2016:   0%|          | 0/5149 [00:00<?, ?it/s]

Movies from 2017:   0%|          | 0/5517 [00:00<?, ?it/s]

Movies from 2018:   0%|          | 0/5637 [00:00<?, ?it/s]

Movies from 2019:   0%|          | 0/5726 [00:00<?, ?it/s]